# Collect all Paths 
This does not need to be ran anymore we have the list in paths.txt now. Just read that into simulation_paths and you are good to go with collecting data

In [283]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd
import pickle
from pprint import pprint
import os
from dotenv import load_dotenv
load_dotenv()

True

In [284]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = os.getenv("ACCESS_KEY")
secret_key = os.getenv("SECRET_KEY")

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'

root = list(fs.ls(bucket))

simulation_paths = []
print("successfully authenticated")

successfully authenticated


In [285]:
def get_paths(rangestart,rangeend):
    global fs, bucket
    paths = fs.ls(bucket)
    for path in paths[rangestart:rangeend]:
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

def read_paths():
    paths = []
    with open("paths.txt","r") as f:
        paths = f.read().splitlines() 
    return paths


In [286]:
# with open("paths.txt", "a") as file:
#     for i in simulation_paths:
#         file.write(i)
#         file.write("\n")

In [294]:
simulation_paths = read_paths()
len(simulation_paths)

26997

# Now Extract Data

In [328]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'time_scraped': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    'dz':lambda d: d['dz'],
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df(count):
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys())
    time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    i = 0
    for p in tqdm(simulation_paths[:count]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
                print(data)
                import sys; sys.exit()
        except:
            filenotfound.append(p)
            print("FileNotFound error on path {",p,"}")
            continue
            
        row = []
        for k,expr in KEEP_ATTRIBUTES.items():
            try:
                value = KEEP_ATTRIBUTES[k](data)
            except KeyError:
                value = None
            row.append(value)
        row[0] = p
        row[1] = time
        df.loc[i] = row
        i+=1

    return df

pd.set_option('display.max_columns', None)
print(filenotfound)

[]


In [329]:
simulation_paths = read_paths()
df = get_df(100)

  1%|▍                                       | 1/100 [00:00<00:59,  1.66it/s]

{'canopy_moisture': 1.0, 'dimension': ['t', 'z', 'y', 'x'], 'dz': None, 'extent': [-1019.635396216905, 587.5323697931051, 780.364603783095, -512.4676302068949], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'extent_origin': 'bottom left', 'fire_grid': None, 'fuel': {'density': None, 'dx': 2, 'dy': 2, 'height': 0.5, 'id': '2a4900b5-dec0-46f3-9bd8-025111b24429', 'xlen': None, 'ylen': None}, 'ignition': {'dat': ['       igntype=    4\n', '    &aeriallist\n', '       naerial=  2384\n', '    targettemp= 1000.00\n', '      ramprate=  172.00\n', '/\n', '516   313   1\n', '516   313   1\n', '514   314   2\n', '517   312   2\n', '519   310   3\n', '513   316   3\n', '520   309   5\n', '512   318   5\n', '511   319   6\n', '522   308   6\n', '510   321   7\n', '523   306   7\n', '509   323   9\n', '525   305   9\n', '507   324   10\n', '526   304   10\n', '506   326   11\n', '528   302   11\n', '529   301   13\n', '505   327   13\n', '531   300   14\n', '504   329   14\n', '503   331   15\n', '532   299

  3%|█▏                                      | 3/100 [00:00<00:26,  3.71it/s]

{'_ARRAY_DIMENSIONS': ['t', 'z', 'y', 'x'], 'canopy_moisture': 1.0, 'dimension': ['t', 'z', 'y', 'x'], 'dz': None, 'extent': [-2048049, 1866939, -2046749, 1866139], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'extent_origin': 'bottom left', 'fire_grid': None, 'fuel': {'density': None, 'dx': 2, 'dy': 2, 'height': 0.5, 'id': 'e9d977fd-94e5-4934-bfe5-dcdfd1f72517', 'xlen': None, 'ylen': None}, 'ignition': {'dat': ['       igntype=    4\n', '    &aeriallist\n', '       naerial=  1943\n', '    targettemp= 1000.00\n', '      ramprate=  172.00\n', '/\n', '560   235   1\n', '560   237   2\n', '560   239   3\n', '560   241   4\n', '560   243   6\n', '560   245   7\n', '560   247   8\n', '560   249   10\n', '559   251   11\n', '559   253   12\n', '558   255   14\n', '557   256   15\n', '557   258   16\n', '556   260   18\n', '555   262   19\n', '553   263   20\n', '552   265   22\n', '551   267   23\n', '550   268   24\n', '549   270   26\n', '548   272   27\n', '547   273   28\n', '546   275   30\n',

  6%|██▍                                     | 6/100 [00:21<05:41,  3.63s/it]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

 10%|███▉                                   | 10/100 [00:21<02:04,  1.38s/it]

...

{'canopy_moisture': 1, 'dimension': ['t', 'z', 'y', 'x'], 'dz': None, 'extent': [1120178.154045004, 913143.8979546748, 1120778.154045004, 912543.8979546748], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'extent_origin': 'bottom left', 'fire_grid': None, 'fuel': {'density': None, 'dx': 2, 'dy': 2, 'height': 0.5, 'id': '76cab8a3-c7c0-43fd-bba5-daff26e5d772', 'xlen': None, 'ylen': None}, 'ignition': {'dat': ['       igntype=    5\n', '       &atvlist\n', '          natv=    2\n', '    targettemp= 1000.00\n', ' flamedistance=    4.00\n', '/ \n', '  100.00  300.00  100.00 400.00    1.0  31.0\n', '  100.00  300.00  100.00 200.00    1.0  31.0'], 'perc': 100}, 'output': {'emissions': False, 'energy_atmos': False, 'fire_energy': False, 'fuels_dens': True, 'fuels_moist': False, 'plume_locs': False, 'plume_traj': False, 'qu_avg_wind': False, 'qu_wind': False, 'reaction_rate': False, 'steps_fire': 10, 'steps_wind': 100, 'thermal_rad': False, 'vert_mass_burnt': False, 'wind': True}, 'resolution': [1, 2, 2

 98%|██████████████████████████████████████▏| 98/100 [01:35<00:00, 11.27it/s]

{'canopy_moisture': 1.0, 'dz': None, 'extent': [-1019.635396216905, 587.5323697931051, 780.364603783095, -512.4676302068949], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'fire_grid': None, 'fuel': {'density': None, 'dx': 2, 'dy': 2, 'height': 0.5, 'id': 'd552fc3c-6a74-4817-9c97-21d9a0c8fad7', 'xlen': None, 'ylen': None}, 'ignition': {'dat': ['       igntype=    4\n', '    &aeriallist\n', '       naerial=  593\n', '    targettemp= 1000.00\n', '      ramprate=  172.00\n', '/\n', '787   185   61\n', '787   184   64\n', '785   175   68\n', '781   167   72\n', '772   165   75\n', '763   164   79\n', '755   169   83\n', '746   171   86\n', '736   173   90\n', '877   10   91\n', '872   17   94\n', '727   173   94\n', '865   23   97\n', '719   168   97\n', '859   31   100\n', '710   166   101\n', '852   37   103\n', '704   159   105\n', '847   45   106\n', '702   157   108\n', '842   53   109\n', '700   148   112\n', '837   61   112\n', '832   69   115\n', '691   149   116\n', '827   77   118\n', '6

100%|██████████████████████████████████████| 100/100 [01:36<00:00,  1.04it/s]

{'canopy_moisture': 1.0, 'dz': None, 'extent': [-1019.635396216905, 587.5323697931051, 780.364603783095, -512.4676302068949], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'fire_grid': None, 'fuel': {'density': None, 'dx': 2, 'dy': 2, 'height': 0.5, 'id': '6aa051fe-f62e-40d3-a138-0d9d5f6cc7a5', 'xlen': None, 'ylen': None}, 'ignition': {'dat': ['       igntype=    4\n', '    &aeriallist\n', '       naerial=  1123\n', '    targettemp= 1000.00\n', '      ramprate=  172.00\n', '/\n', '516   313   1\n', '516   313   1\n', '514   314   2\n', '517   312   2\n', '519   310   3\n', '513   316   3\n', '520   309   5\n', '512   318   5\n', '511   319   6\n', '522   308   6\n', '510   321   7\n', '523   306   7\n', '509   323   9\n', '525   305   9\n', '507   324   10\n', '526   304   10\n', '506   326   11\n', '528   302   11\n', '529   301   13\n', '505   327   13\n', '531   300   14\n', '504   329   14\n', '503   331   15\n', '532   299   15\n', '534   297   17\n', '502   332   17\n', '500   334   18\n

In [327]:
df.head()

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
start,,,,,,,,,,,,,,,,,,,,,
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,6772472,-1,3200,0.35,8,25,None,225.0,3.0
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0b/run_000baafc-fe1a-48a1-90c5-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-2048049, 1866939, -2046749, 1866139]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,3561948,-1,2100,0.35,None,25,None,180.0,8.0
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-...,2023-07-27 14:22:05.476561+00:00,1.5,None,"[939794.0, 1235562.0, 941860.0, 1233570.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1.0, 2.0, 2.0]",meters,/quicfire/quicfire-v4.0.1,2023-07-27 14:22:05.476561+00:00,None,-1,18600,0.05,1,None,None,245.0,1.0
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000de038-3005-461e-a6d6-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,428896,-1,600,0.35,1,100,{'total_startup_iters': 0},270.0,6.0
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0e/run_000e95ef-a111-4371-a698-...,2023-07-27 14:22:05.476561+00:00,120.0,None,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,None,-1,15963,7.00,1,None,None,335.0,3.0


## remove NaN rows

In [312]:
df = df.dropna(subset=['run_end','run_start','sim_time'])

# df2 = df[['sim_time', 'run_binary', 'run_end', 'run_start', 'wind_direction','extent']]
# print(df2.head())
df = df.rename(columns={"run_end": "stop", "run_start": "start"})
df = df.drop(columns=["ignition","fuel"], axis=1)
df['time_scraped'] = pd.to_datetime(df['time_scraped'], utc=True)
df['start'] = pd.to_datetime(df['time_scraped'], utc=True)
df['stop'] = pd.to_datetime(df['time_scraped'], utc=True)

df = df.set_index("start")
df

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
start,,,,,,,,,,,,,,,,,,,,,
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,6772472,-1,3200,0.35,8,25,None,225.000000,3.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0b/run_000baafc-fe1a-48a1-90c5-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-2048049, 1866939, -2046749, 1866139]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,3561948,-1,2100,0.35,None,25,None,180.000000,8.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-...,2023-07-27 14:22:05.476561+00:00,1.5,None,"[939794.0, 1235562.0, 941860.0, 1233570.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1.0, 2.0, 2.0]",meters,/quicfire/quicfire-v4.0.1,2023-07-27 14:22:05.476561+00:00,None,-1,18600,0.05,1,None,None,245.000000,1.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000de038-3005-461e-a6d6-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,428896,-1,600,0.35,1,100,{'total_startup_iters': 0},270.000000,6.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0e/run_000e95ef-a111-4371-a698-...,2023-07-27 14:22:05.476561+00:00,120.0,None,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,None,-1,15963,7.00,1,None,None,335.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/02/run_0102f75c-8d04-46b6-86ea-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[1120178.154045004, 913143.8979546748, 1120778...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,523068,-1,1000,0.00,1,10,None,264.614715,4.550601
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/06/run_01062a0e-e7ee-4420-aea2-...,2023-07-27 14:22:05.476561+00:00,0.1,None,"[0, 500, 500, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.0.0,2023-07-27 14:22:05.476561+00:00,301760,-1,5,0.05,1,100,None,270.000000,5.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/08/run_010827f3-e59d-422e-9f42-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': True, 'fi...","[2, 2]",meters,/ceph/QF_Versions/QF_1-6-22/source_code/compil...,2023-07-27 14:22:05.476561+00:00,654512,-1,600,0.10,None,1,{'total_startup_iters': 0},280.000000,10.000000


# Push Data to Influx

In [313]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient, Point, WriteOptions

bucket = "devin"
url= "http://localhost:8086"
token = os.getenv("INFLUXDB_TOKEN")
org = "SDSC internship"
bucket = "perf-data-testing"

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Write script
write_api = client.write_api(write_options=SYNCHRONOUS)

# p = influxdb_client.Point("hi").tag("location", "Czech").field("height",6).field("age",8)
# write_api.write(bucket=bucket, org=org, record=p)

In [314]:
'2023-02-24T11:37:45.285185-08:00'
datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")

'2023-07-27T14:22:56.001760'

In [315]:

# df

# df = df.drop(columns=['ignition','fuel'])
# df
# DataFrame must have the timestamp column as an index for the client. 


write_api.write(bucket, record=df, data_frame_measurement_name='bp3d-run',
                    data_frame_tag_columns=[])

In [316]:
df

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
start,,,,,,,,,,,,,,,,,,,,,
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,6772472,-1,3200,0.35,8,25,None,225.000000,3.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0b/run_000baafc-fe1a-48a1-90c5-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-2048049, 1866939, -2046749, 1866139]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,3561948,-1,2100,0.35,None,25,None,180.000000,8.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-...,2023-07-27 14:22:05.476561+00:00,1.5,None,"[939794.0, 1235562.0, 941860.0, 1233570.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1.0, 2.0, 2.0]",meters,/quicfire/quicfire-v4.0.1,2023-07-27 14:22:05.476561+00:00,None,-1,18600,0.05,1,None,None,245.000000,1.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000de038-3005-461e-a6d6-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,428896,-1,600,0.35,1,100,{'total_startup_iters': 0},270.000000,6.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0e/run_000e95ef-a111-4371-a698-...,2023-07-27 14:22:05.476561+00:00,120.0,None,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,None,-1,15963,7.00,1,None,None,335.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/02/run_0102f75c-8d04-46b6-86ea-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[1120178.154045004, 913143.8979546748, 1120778...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,523068,-1,1000,0.00,1,10,None,264.614715,4.550601
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/06/run_01062a0e-e7ee-4420-aea2-...,2023-07-27 14:22:05.476561+00:00,0.1,None,"[0, 500, 500, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.0.0,2023-07-27 14:22:05.476561+00:00,301760,-1,5,0.05,1,100,None,270.000000,5.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/08/run_010827f3-e59d-422e-9f42-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': True, 'fi...","[2, 2]",meters,/ceph/QF_Versions/QF_1-6-22/source_code/compil...,2023-07-27 14:22:05.476561+00:00,654512,-1,600,0.10,None,1,{'total_startup_iters': 0},280.000000,10.000000


# Now Query for the Data we entered

In [321]:
query_api = client.query_api()

query= '''
from(bucket: "perf-data-testing")
|> range(start:-5d, stop: now())
|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
|> group(columns: ["fuel"])
'''

data = client.query_api().query_data_frame(org=org, query=query)
data

,result,table,_start,_stop,_time,_measurement,canopy_moisture,dz,extent,extent_fmt,fire_grid,fuel,ignition,output,path,resolution,resolution_units,run_binary,run_max_mem_rss_bytes,seed,sim_time,stop,surface_moisture,threads,time_scraped,timestep,topo,wind_direction,wind_speed
0,_result,0,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:22:05.476561+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:22:05.476561+00:00,0.35,8,2023-07-27 14:22:05.476561+00:00,None,None,195.0,2.0
1,_result,1,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:16:00.039445+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4\n', ' &aeria...","{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:16:00.039445+00:00,0.35,8,2023-07-27 14:16:00.039445+00:00,None,None,195.0,2.0


In [324]:
data

,result,table,_start,_stop,_time,_measurement,canopy_moisture,dz,extent,extent_fmt,fire_grid,fuel,ignition,output,path,resolution,resolution_units,run_binary,run_max_mem_rss_bytes,seed,sim_time,stop,surface_moisture,threads,time_scraped,timestep,topo,wind_direction,wind_speed
0,_result,0,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:22:05.476561+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:22:05.476561+00:00,0.35,8,2023-07-27 14:22:05.476561+00:00,None,None,195.0,2.0
1,_result,1,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:16:00.039445+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4\n', ' &aeria...","{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:16:00.039445+00:00,0.35,8,2023-07-27 14:16:00.039445+00:00,None,None,195.0,2.0


In [322]:
LATEST_VERSION = 'quicfire-v5.2-ucsd-rel'
# drop all multi threaded and none rows
# keep a record of how many left


In [ ]:
[-2048049, 1866939, -2046749, 1866139]
[x1 y1 x2 y2]
A = (x1-x2) * (y1-y2)

# resolution
[1, 2, 2]
[ z x y ]